In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 视频仓库 SDK 演示

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/vision/video_warehouse_sdk.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/vision/video_warehouse_sdk.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>                                                                                         
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/vision/video_warehouse_sdk.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI Workbench 中打开
    </a>
  </td>
</table>

**_注意_**：此笔记本在以下环境中进行了测试：

* Python版本 = 3.10

## 概述

通过使用SDK逐步学习如何构建一个[视频仓库](https://cloud.google.com/vision-ai/docs)。

### 目标
本示例旨在展示如何使用Warehouse SDK处理输入视频，建立索引并进行搜索。

执行步骤包括:

* 创建语料库。

* 使用来自Google Cloud Storage的视频文件创建和上传资产。

* 创建索引，创建索引端点，并部署索引。
  * 此步骤可能需要一个小时。

* 运行转换以分析资产:
  * 语音转换
    * 使用Video intelligence API运行语音转录并存储到仓库中。默认情况下，语音结果可以通过"speech"字段进行搜索。您可以通过在SpeechTransformerInitConfig中设置speech_transcript_search_criteria_key来指定搜索条件字段。
  * OCR转换
    * 使用Video intelligence API运行文本检测并存储到仓库中。默认情况下，文本检测结果可以通过"text"字段进行搜索。通过在OcrTransformerInitConfig中设置ocr_search_criteria_key来指定搜索条件字段。
  * 嵌入式分析

* 建立索引。

* 搜索。

* 清理资源（资产、索引、索引端点、语料库）。

### 数据集
该数据集将使用存储在Google Cloud Storage桶中的视频集合：[gs://cloud-samples-data/video](https://pantheon.corp.google.com/storage/browser/cloud-samples-data/video)。

本教程演示了如何使用Warehouse SDK来运行语音转录、文本检测、嵌入式分析，以及为该数据集中的视频构建搜索索引并提供搜索功能。

### 成本

本教程使用了 Google Cloud 的计费组件：

Vertex AI Vision（[定价](https://cloud.google.com/vision-ai/pricing)）

Video Intelligence（[定价](https://cloud.google.com/video-intelligence/pricing)）

## 安装

安装以下所需的软件包以执行这个笔记本。

In [ ]:
!gsutil cp gs://visionai-artifacts/visionai-0.0.6-py3-none-any.whl .
!pip install visionai-0.0.6-py3-none-any.whl --force-reinstall

只有Colab才能：取消对以下单元格的注释以重新启动内核。

In [ ]:
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 开始之前

### 设置你的Google云项目

**无论你使用什么笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个Google云项目](https://console.cloud.google.com/cloud-resource-manager)。当你第一次创建账户时，你会获得$300的免费信用额度用于计算/存储成本。

2. [确保你的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[找到项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

### 验证您的Google Cloud帐户

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照以下相关说明进行操作。

1. Vertex AI Workbench
* 无需操作，因为您已经验证过了。

2. 本地 JupyterLab 实例，取消注释并运行：

In [ ]:
# ! gcloud auth login

In [ ]:
# ! gcloud auth application-default login

3. Colab，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user(project_id=PROJECT_ID)

### 设置其他常量

In [ ]:
PROJECT_NUMBER_STR = !gcloud projects describe $PROJECT_ID --format="value(projectNumber)"
PROJECT_NUMBER = int(PROJECT_NUMBER_STR[0])

# Only us-central1 is supported.
# Please note that this region is for VisionAi services. For speech
# transcription, we may not respect the region here.
REGION = "us-central1"

CORPUS_DISPLAY_NAME = "Demo corpus"  # @param {type: "string"}
CORPUS_DESCRIPTION = "Demo corpus to demo warehouse transformations and search"  # @param {type: "string"}

# External users can only access PROD environment.
ENV = "PROD"

INDEX_DISPLAY_NAME = "Demo Index"  # @param {type: "string"}
INDEX_ENDPOINT_DISPLAY_NAME = "Demo Index Endpoint"  # @param {type: "string"}

CLEAN_UP_ASSETS = True  # @param {type: "boolean"}
CLEAN_UP_INDEX = True  # @param {type: "boolean"}
CLEAN_UP_CORPUS = True  # @param {type: "boolean"}

无论使用现有语料库和索引

In [ ]:
# Because it takes ~1h to create and deploy index. A existing index can be
# specified to save time.

# If CORPUS_ID is specified, skip creating a new corpus.
CORPUS_ID = None  # @param {type: "string"}
# If DEPLOYED_INDEX_ID is specified, use existing index instead of creating and
# deploying a new index.
DEPLOYED_INDEX_ID = None  # @param {type: "string"}

### 输入视频文件

In [ ]:
GCS_FILES = [
    "gs://cloud-samples-data/video/animals.mp4",
    "gs://cloud-samples-data/video/googlework_short.mp4",
    "gs://cloud-samples-data/video/chicago.mp4",
    (
        "gs://cloud-samples-data/video/Machine Learning Solving Problems"
        " Big, Small, and Prickly.mp4"
    ),
    "gs://cloud-samples-data/video/JaneGoodall.mp4",
    "gs://cloud-samples-data/video/gbikes_dinosaur.mp4",
    "gs://cloud-samples-data/video/pizza.mp4",
]

### 启用 API

In [ ]:
!gcloud services enable videointelligence.googleapis.com
!gcloud services enable visionai.googleapis.com

### 配置日志

In [ ]:
import logging

logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)
_logger = logging.getLogger("colab")

### 导入库

In [ ]:
import concurrent
import logging

from visionai.python.gapic.visionai import visionai_v1
from visionai.python.net import channel
from visionai.python.warehouse.transformer import \
    asset_indexing_transformer as ait
from visionai.python.warehouse.transformer import (ocr_transformer,
                                                   speech_transformer,
                                                   transformer_factory)
from visionai.python.warehouse.utils import (vod_asset, vod_corpus,
                                             vod_index_endpoint)

## 创建仓库客户端

In [ ]:
warehouse_endpoint = channel.get_warehouse_service_endpoint(channel.Environment[ENV])
warehouse_client = visionai_v1.WarehouseClient(
    client_options={"api_endpoint": warehouse_endpoint}
)

创建一个语料库或使用现有的语料库。

In [ ]:
if CORPUS_ID is None:
    corpus_name = vod_corpus.create_corpus(
        warehouse_client,
        PROJECT_NUMBER,
        REGION,
        CORPUS_DISPLAY_NAME,
        CORPUS_DESCRIPTION,
    ).name
else:
    corpus_name = visionai_v1.WarehouseClient.corpus_path(
        PROJECT_NUMBER, REGION, CORPUS_ID
    )

创建一个执行者

In [ ]:
# Creates an executor to upload and transform assets in parallel.
executor = concurrent.futures.ThreadPoolExecutor(max_workers=8)

创建和上传资源

In [ ]:
new_asset_futures = []
for gcs_file in GCS_FILES:
    new_asset_futures.append(
        executor.submit(
            vod_asset.create_and_upload_asset,
            warehouse_client,
            gcs_file,
            corpus_name,
        )
    )
done_or_error, _ = concurrent.futures.wait(
    new_asset_futures, return_when="ALL_COMPLETED"
)
asset_names = []
for done_future in done_or_error:
    try:
        asset_names.append(done_future.result())
        _logger.info("Create and upload asset succeeded %s", done_future.result())
    except Exception as e:
        _logger.exception(e)

准备索引或使用现有索引

In [ ]:
# Create index and index endpoint for the corpus, or use existing index
# and index endpoint if specified.
if DEPLOYED_INDEX_ID is None:
    # Creates index for the corpus.
    index_name = vod_corpus.index_corpus(
        warehouse_client, corpus_name, INDEX_DISPLAY_NAME
    )
    # Creates index endpoint and deploys the created index above to the index
    # endpoint.
    index_endpoint_name = vod_index_endpoint.create_index_endpoint(
        warehouse_client,
        PROJECT_NUMBER,
        REGION,
        INDEX_ENDPOINT_DISPLAY_NAME,
    ).name
    deploy_operation = warehouse_client.deploy_index(
        visionai_v1.DeployIndexRequest(
            index_endpoint=index_endpoint_name,
            deployed_index=visionai_v1.DeployedIndex(
                index=index_name,
            ),
        )
    )
    _logger.info("Wait for index to be deployed %s.", deploy_operation.operation.name)
    # Wait for the deploy index operation. Depends on the data size to be
    # indexed, the timeout may need to be increased.
    deploy_operation.result(timeout=7200)
    _logger.info("Index is deployed.")
else:
    index_name = "{}/indexes/{}".format(corpus_name, DEPLOYED_INDEX_ID)
    index = warehouse_client.get_index(visionai_v1.GetIndexRequest(name=index_name))
    _logger.info("Use existing index %s.", index)
    if index.state != visionai_v1.Index.State.CREATED:
        _logger.critical("Invalid index. The index state must be Created.")
    if not index.deployed_indexes:
        _logger.critical("Invalid index. The index must be deployed.")
    index_endpoint_name = index.deployed_indexes[0].index_endpoint

## 运行转换

In [ ]:
ocr_config = ocr_transformer.OcrTransformerInitConfig(
    corpus_name=corpus_name,
    env=channel.Environment[ENV],
)

ml_config = transformer_factory.MlTransformersCreationConfig(
    run_embedding=True,
    speech_transformer_init_config=speech_transformer.SpeechTransformerInitConfig(
        corpus_name=corpus_name, language_code="en-US"
    ),
    ocr_transformer_init_config=ocr_config,
)
ml_transformers = transformer_factory.create_ml_transformers(
    warehouse_client, ml_config
)
# Creates indexing transformer to index assets.
asset_indexing_transformer = ait.AssetIndexingTransformer(warehouse_client, index_name)
# Runs the transformers for the assets.
futures = []

for asset_name in asset_names:
    futures.append(
        executor.submit(
            vod_asset.transform_single_asset,
            asset_name,
            ml_transformers,
            asset_indexing_transformer,
        )
    )
done_or_error, _ = concurrent.futures.wait(futures, return_when="ALL_COMPLETED")
for future in done_or_error:
    try:
        future.result()
    except Exception as e:
        _logger.exception(e)

all_transformers = ml_transformers + [asset_indexing_transformer]
for transformer in all_transformers:
    transformer.teardown()

搜索

In [ ]:
search_response = warehouse_client.search_index_endpoint(
    visionai_v1.SearchIndexEndpointRequest(
        index_endpoint=index_endpoint_name,
        text_query="dinosaur",
        page_size=10,
    )
)
_logger.info("Search response: %s", search_response)

In [ ]:
cr = visionai_v1.Criteria(
    field="speech", text_array=visionai_v1.StringArray(txt_values=["kid"])
)
search_response = warehouse_client.search_index_endpoint(
    visionai_v1.SearchIndexEndpointRequest(
        index_endpoint=index_endpoint_name,
        text_query="river",
        criteria=[cr],
        page_size=100,
    )
)
_logger.info("Search response: %s", search_response)

In [ ]:
cr = visionai_v1.Criteria(
    field="text", text_array=visionai_v1.StringArray(txt_values=["National Park"])
)
search_response = warehouse_client.search_index_endpoint(
    visionai_v1.SearchIndexEndpointRequest(
        index_endpoint=index_endpoint_name,
        text_query="trees",
        criteria=[cr],
        page_size=100,
    )
)
_logger.info("Search response: %s", search_response)

清理

In [ ]:
if CLEAN_UP_ASSETS:
    for asset_name in asset_names:
        warehouse_client.delete_asset(visionai_v1.DeleteAssetRequest(name=asset_name))
        _logger.info("Deleted asset %s", asset_name)

if CLEAN_UP_INDEX:
    undeploy_operation = warehouse_client.undeploy_index(
        visionai_v1.UndeployIndexRequest(index_endpoint=index_endpoint_name)
    )
    _logger.info(
        "Wait for index to be undeployed %s.",
        undeploy_operation.operation.name,
    )
    # Wait for the undeploy index operation.
    undeploy_operation.result(timeout=1800)
    _logger.info("Index is undeployed.")
    warehouse_client.delete_index(visionai_v1.DeleteIndexRequest(name=index_name))
    _logger.info("Deleted index %s", index_name)
    warehouse_client.delete_index_endpoint(
        visionai_v1.DeleteIndexEndpointRequest(name=index_endpoint_name)
    )
    _logger.info("Deleted index endpoint %s", index_endpoint_name)

if CLEAN_UP_CORPUS:
    warehouse_client.delete_corpus(visionai_v1.DeleteCorpusRequest(name=corpus_name))
    _logger.info("Deleted corpus %s", corpus_name)